In [2]:
import json
import re
from collections import Counter
import inflect
import random

In [3]:
random.seed(42)

all_train = json.load(open('raw_data/all_train.json',encoding='utf-8'))
test = json.load(open('raw_data/test_set_new.json',encoding='utf-8'))
all = all_train + test
no_task = [x for x in all if x.get('task_name', None) == None]
all = [x for x in all if x.get('task_name', None) != None]

sign_dict = {'red': 'negative', 'green': 'positive', 'yellow': 'negligible'}

for i in range(len(all)):
    # Some of the data is in string form, eval() is to convert it to dict
    try:
        all[i]['feature_division'] = eval(all[i]['feature_division'])
    except:
        all[i]['feature_division'] = all[i]['feature_division']
    all[i]['feature_division']['explainable_df'] = eval(all[i]['feature_division']['explainable_df'])
    
    # Some of the fields we want are inside the feature_division dict, moving them to the top level
    all[i]['values'] = [format(val, '.2f') for val in all[i]['feature_division']['explainable_df']['Values'].values()]
    ft_nums =[re.search('F\d*', val).group() for val in list(all[i]['feature_division']['explainable_df']['annotate_placeholder'].values())]
    ft_names = list(all[i]['feature_division']['explainable_df']['Variable'].values())
    all[i]['sign'] = [sign_dict[x] for x in all[i]['feature_division']['explainable_df']['Sign'].values()]
    all[i]['narrative_id'] = all[i].pop('id')
    all[i]['unique_id'] = i
    all[i]['classes_dict'] = {v[0].strip(): v[1].strip() for v in [y.split(':') for y in [x for x in all[i]['prediction_confidence_level'].split(',')]]}
    all[i]['narrative_questions'] = all[i]['narrative_question'].strip('<ul><li>/ ').split(' </li> <li> ')
    
    # Shuffle feature names to ensure separation of train and test
    new_ft_nums = ft_nums.copy()
    random.shuffle(new_ft_nums)
    old2new_ft_nums = dict(zip(ft_nums, new_ft_nums))
    ft_ptn = re.compile("|".join([f'{k}\\b' for k in old2new_ft_nums.keys()]))
    
    all[i]['feature_nums'] = new_ft_nums
    all[i]['ft_num_to_name'] = str(dict(zip(new_ft_nums, ft_names)))
    all[i]['old2new_ft_nums'] = str(old2new_ft_nums)
    all[i]['narration'] = ft_ptn.sub(lambda m: old2new_ft_nums[re.escape(m.group(0))], all[i]['narration'])
    all[i]['narrative_questions'] = [ft_ptn.sub(lambda m: old2new_ft_nums[re.escape(m.group(0))], x) for x in all[i]['narrative_questions']]
    
    # # Shuffle class names too
    new_classes = list(all[i]['classes_dict'].keys()).copy()
    random.shuffle(new_classes)
    old2new_classes = dict(zip(list(all[i]['classes_dict'].keys()), new_classes))
    cls_ptn = re.compile("|".join([f'{k}\\b' for k in old2new_classes.keys()]))
    
    all[i]['predicted_class'] = cls_ptn.sub(lambda m: old2new_classes[re.escape(m.group(0))], all[i]['predicted_class'])
    all[i]['narration'] = cls_ptn.sub(lambda m: old2new_classes[re.escape(m.group(0))], all[i]['narration'])
    all[i]['classes_dict'] = str({old2new_classes[k]: v for k, v in all[i]['classes_dict'].items()})
    all[i]['narrative_questions'] = [cls_ptn.sub(lambda m: old2new_classes[re.escape(m.group(0))], x) for x in all[i]['narrative_questions']]
    all[i]['old2new_classes'] = str(old2new_classes)

    
    for key in ['deleted', 'mturk_id','narrative_status', 'predicted_class_label', 'date_submitted',
                'date_approved', 'features_placeholder', 'is_paid', 'redeem_code', 'narrator',
                'user_ip','feature_division', 'narrative_question', 'prediction_confidence_level',
                'test_instance', "prediction_confidence"]:
        try:
            all[i].pop(key)
        except:
            pass
# no_task = prepare_all(no_task)
json.dump(all, open('jb_data/all.json', 'w', encoding='utf-8'), indent=4)

# Split into train, test, val 80:10:10
random.shuffle(all)
train = all[:int(0.8*len(all))]
test = all[int(0.8*len(all)):int(0.9*len(all))]
val = all[int(0.9*len(all)):]

json.dump(train, open('jb_data/train.json', 'w', encoding='utf-8'), indent=4)
json.dump(test, open('jb_data/test.json', 'w', encoding='utf-8'), indent=4)
json.dump(val, open('jb_data/val.json', 'w', encoding='utf-8'), indent=4)


In [3]:
train[4]

{'model_name': 'RandomForestClassifier',
 'predicted_class': 'C1',
 'task_name': 'Used Cars Price-Range Prediction',
 'narration': "Per the model, class C2 has a prediction probability of 10.50 percent, whereas class C1 has a predicted probability of 89.50 percent. As a result of the model, it can be determined that C1 is the most likely label for the given scenario. All of the input features are shown to contribute to the above conclusion, with F1, F7, and F3 having the most influence on the classification decision. The least influential features with regard to this classification are F5, F2, F10, and F4, whereas, the impact of F8, F6, and F9 can be classified as modest. The large positive contributions of F7 and F1 are responsible for the model's high confidence which further supported by the positive contributions of F8, F5, and F2. In conclusion, the negative features F3, F6, F10, F9, and F4 favour labelling the case as C2 hence the associated predicted probability.",
 'values': ['

# Linearisation

Essel is:

For non-step wise

    'prediction: predictionlabel && predictionlabel: 51.62% && predictionrankB: 48.38% <|section-sep|> featAN featCP featDP featEN featFP featGP featHP featIP <|section-sep|> <mentions> featCP featDP featFP featGP featHP featIP <|section-sep|> featAN featEN <|section-sep|> </mentions> <|section-sep|> <explain> [N0S]'

Special tokens are:

    additional_vocab = new_tokens = ['[EON]', '[NLS]', '[N9S]',
                                        '[N10S]', '[PFS]', '[NFS]', '[IFS]',
                                        '[N4S]', '[N5S]', '[N8S]', '[N6S]',
                                        '[N7S]', '[N1S]', '[N2S]','[CON]',
                                        '[N0S]', '[N3S]']  # +['predictionlabel', 'predictionrankA','predictionrankB', 'predictionrankC', 'predictionrankD', 'predictionrankE']
    specials = ['<positives>', '<neutrals>', '<negatives>',
                '</positives>', '</negatives>', '</neutrals>',
                '<mentions>', "</mentions>"]
    special_tokens = ['&&',
                        '<|>',
                        '<full_explain>',
                        '<full_narration>',
                        "<next_sequence>",
                        "</next_sequence>",
                        '<explain>']+specials



In [9]:
'| predicted class | C4 100.00% | other classes | C2 0.00%; C3 0.00%; C1 0.00% |'


other_classes = "&& ".join([f"{k} {v}" for k,v in all[0]['classes_dict'].items() if k != all[0]["predicted_class"]])
just_fts = ' '.join(all[0]['feature_nums'])
fts_and_signs = "&& ".join([f'{a} {b} ' for a, b in zip(all[0]['feature_nums'], all[0]['sign'])])
fts_and_pos = "&& ".join([f'{a} {b} ' for a, b in zip(all[0]['feature_nums'], all[0]['sign']) if b == 'positive'])
fts_and_nega = "&& ".join([f'{a} {b} ' for a, b in zip(all[0]['feature_nums'], all[0]['sign']) if b == 'negative'])
fts_and_negl = "&& ".join([f'{a} {b} ' for a, b in zip(all[0]['feature_nums'], all[0]['sign']) if b == 'negligible'])
fts_and_negl = 'None' if fts_and_negl == '' else fts_and_negl

essel_input = f'| predicted class | {all[0]["predicted_class"]} {all[0]["classes_dict"][all[0]["predicted_class"]]} | other classes | {other_classes} | \
features | {fts_and_signs}| postive features | {fts_and_pos} | negative features | {fts_and_nega} | \
negligible features | {fts_and_negl} |'

p = inflect.engine()

ordinals = [p.ordinal(i+1) for i in range(len(all[0]['feature_nums']))]

features = ' '.join([f'| {o} | {f} {s} {v}' for o, f, s, v in zip(ordinals, all[0]['feature_nums'], all[0]['sign'], all[0]['values'])])
    
ord_first_input = f'| predicted class | {all[0]["predicted_class"]} {all[0]["classes_dict"][all[0]["predicted_class"]]} | other classes | {other_classes} {features} |'

ft_first_input = ' '.join([f'| {f} | {o} {s} {v}' for o, f, s, v in zip(ordinals, all[0]['feature_nums'], all[0]['sign'], all[0]['values'])])

print(ft_first_input)

| F5 | 1st positive 0.05 | F8 | 2nd positive 0.03 | F1 | 3rd positive 0.03 | F11 | 4th negative -0.03 | F7 | 5th positive 0.02 | F6 | 6th negative -0.02 | F16 | 7th positive 0.02 | F15 | 8th positive 0.02 | F13 | 9th negative -0.02 | F3 | 10th positive 0.02 | F14 | 11th positive 0.01 | F9 | 12th positive 0.01 | F10 | 13th positive 0.00 | F2 | 14th negative -0.00 | F12 | 15th negative -0.00 | F4 | 16th negative -0.00


In [22]:
print('\n'.join([f'{idx+1}. {q}' for idx, q in enumerate(all[0]['narrative_questions'])]))

1. For this test instance, provide information on the predicted label along with the confidence level of the model's decision.
2. Summarize the top features influencing the model's decision along with the respective directions of influence on the prediction?
3. Summarize the direction of influence of the features (F6, F16 and F15) with moderate impact on the prediction made for this test case.


In [16]:
print(f"{ft_first_input} \n <br> <br> Using the above information, answer the following in detail: <br> <br> 1. For this test instance, provide information on the predicted label along with the confidence level of the model's decision. <br> 2. Summarize the top features influencing the model's decision along with the respective directions of influence on the prediction? <br> 3. Summarize the direction of influence of the features (F6, F16 and F15) with moderate impact on the prediction made for this test case. <explain>")

| F5 | 1st positive 0.05 | F8 | 2nd positive 0.03 | F1 | 3rd positive 0.03 | F11 | 4th negative -0.03 | F7 | 5th positive 0.02 | F6 | 6th negative -0.02 | F16 | 7th positive 0.02 | F15 | 8th positive 0.02 | F13 | 9th negative -0.02 | F3 | 10th positive 0.02 | F14 | 11th positive 0.01 | F9 | 12th positive 0.01 | F10 | 13th positive 0.00 | F2 | 14th negative -0.00 | F12 | 15th negative -0.00 | F4 | 16th negative -0.00 
 <br> <br> Using the above information, answer the following in detail: <br> <br> 1. For this test instance, provide information on the predicted label along with the confidence level of the model's decision. <br> 2. Summarize the top features influencing the model's decision along with the respective directions of influence on the prediction? <br> 3. Summarize the direction of influence of the features (F6, F16 and F15) with moderate impact on the prediction made for this test case. <explain>


In [ ]:
print(ord_first_input)

: 